In [1]:
import duckdb
import pyarrow as pa
import pyarrow.parquet as pq

import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import zipfile
import os
from pathlib import Path

## Create a duckdb connexion

In this tutorial, we use duckdb as in memory db without any persistence. We can easily use the below config to persiste the data in a local file system

```python
conn = duckdb.connect(database="/tmp/my_database.db")
```

In [3]:
# Connexion à DuckDB
conn = duckdb.connect(database=':memory:', read_only=False)

# Exécution d'une requête SQL pour cree une table
cursor = conn.cursor()
cursor.execute("CREATE TABLE sample_table(id INTEGER, name VARCHAR)")
cursor.execute("INSERT INTO sample_table VALUES (1, 'John'), (2, 'Alice'), (3, 'Bob')")

# Récupération des résultats de la requête
result = cursor.execute("SELECT * FROM sample_table").fetchall()
print("Résultat de la requête : ", result)



Résultat de la requête :  [(1, 'John'), (2, 'Alice'), (3, 'Bob')]


## Use duckdb to read parquet file

In [4]:
data_path = Path.cwd().parent / "data"
fr_immo_raw_path = (data_path / "fr_immo_transactions.parquet").as_posix()

In [5]:
# Chargement du fichier Parquet
table_name = "fr_immo"
cursor = conn.cursor()
cursor.execute(f"CREATE TABLE {table_name} AS SELECT * FROM parquet_scan('{fr_immo_raw_path}')")


In [6]:
def show_tables():
    # List all tables in the database
    tables = conn.execute("SHOW TABLES").fetchdf()

    # Print the table names
    print(f"Exsiting table list:\n{tables}")

show_tables()

Exsiting table list:
           name
0       fr_immo
1  sample_table


In [7]:
# Affichage des 5 premières lignes de table immo_fr
result = cursor.execute(f"SELECT * FROM {table_name} LIMIT 5").fetchdf()
print("Les 5 premières lignes du fichier Parquet\n:", result)

Les 5 premières lignes du fichier Parquet
:    id_transaction date_transaction      prix departement  id_ville  \
0          141653       2014-01-02  197000.0          01       427   
1          141970       2014-01-02  157500.0          01       451   
2          139240       2014-01-02  112000.0          01       365   
3          146016       2014-01-02  173020.0          01       202   
4          145911       2014-01-03   88000.0          01       283   

                  ville  code_postal                      adresse  \
0               TREVOUX         1600           6346 MTE DES LILAS   
1                VIRIAT         1440       1369 RTE DE STRASBOURG   
2  SAINT-JEAN-SUR-VEYLE         1290   5174  SAINT JEAN SUR VEYLE   
3               LAGNIEU         1150  21 GR GRANDE RUE DE BULLIEZ   
4               OYONNAX         1100          29B RUE DE LA FORGE   

  type_batiment  n_pieces  surface_habitable   latitude  longitude  
0   Appartement         4                 84  45.94

In [8]:
## Check the schema and basic stats of the table

In [9]:
# Statistiques de base sur les colonnes
table_schema = conn.execute(f"""
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_name = '{table_name}'
""").fetchdf()

print("Schema de table :")
print(f"Table schema:\n{table_schema}")

Schema de table :
Table schema:
          column_name     data_type
0      id_transaction       INTEGER
1    date_transaction  TIMESTAMP_NS
2                prix        DOUBLE
3         departement       VARCHAR
4            id_ville       INTEGER
5               ville       VARCHAR
6         code_postal       INTEGER
7             adresse       VARCHAR
8       type_batiment       VARCHAR
9            n_pieces       INTEGER
10  surface_habitable       INTEGER
11           latitude        DOUBLE
12          longitude        DOUBLE


In [10]:
# Statistiques de base sur les données (pour les colonnes numériques)
numeric_columns_query = f"""
    SELECT column_name
    FROM information_schema.columns
    WHERE table_name = '{table_name}'
    AND data_type IN ('INTEGER', 'DOUBLE', 'FLOAT', 'DECIMAL')
"""
numeric_columns = conn.execute(numeric_columns_query).fetchall()

# Construction de la requête SQL pour les statistiques des colonnes numériques
stats_queries = []
for column in numeric_columns:
    column_name = column[0]
    stats_queries.append(f"""
        SELECT
            '{column_name}' AS column_name,
            AVG({column_name}) AS average,
            STDDEV({column_name}) AS standard_deviation,
            MIN({column_name}) AS min_value,
            MAX({column_name}) AS max_value
        FROM {table_name}
    """)

# Exécution des requêtes de statistiques et affichage des résultats
print("Statistiques des colonnes numériques :")
print("Nom de colonne / Moyenne / Ecart type / Minimum / Maximum")
for query in stats_queries:
    result = conn.execute(query).fetchall()
    print(result)

Statistiques des colonnes numériques :
Nom de colonne / Moyenne / Ecart type / Minimum / Maximum
[('id_transaction', 7739952.499090474, 4446977.8655904615, 1, 15058573)]
[('id_ville', 208.0698140243479, 171.9242239066142, 1, 909)]
[('code_postal', 52936.02979454411, 27696.004797471785, 1000, 97490)]
[('n_pieces', 3.524251132709874, 1.5448404739813182, 0, 112)]
[('surface_habitable', 82.18102442544625, 43.386028549398894, 0, 7626)]
[('prix', 225329.25414140944, 1663532.0597511746, 0.0, 3300000000.0)]
[('latitude', 46.269937361272454, 6.012010692496521, -21.3862188488928, 51.082059485914)]
[('longitude', 2.385461679804203, 6.443977622752839, -63.1520742420684, 55.8292394321594)]


## Q2. Qulle est la ville le plus active sur le marche immobilier?
 

In [11]:


query = f"""
SELECT EXTRACT(YEAR FROM date_transaction) AS year, 
ville, 
COUNT(*) AS transaction_count
FROM {table_name}
GROUP BY year, ville
ORDER BY transaction_count desc;
"""

result = conn.execute(query).fetchdf()

# Print the grouped data
print(f"Nombre de transaction group by year et ville:\n{result}")

Nombre de transaction group by year et ville:
        year          ville  transaction_count
0       2017       TOULOUSE              10791
1       2021       TOULOUSE               9944
2       2022       TOULOUSE               9911
3       2019       TOULOUSE               9881
4       2022           NICE               9431
...      ...            ...                ...
312107  2017        MACOUBA                  1
312108  2020  GRAND RIVIERE                  1
312109  2018           MANA                  1
312110  2020         APATOU                  1
312111  2023    MARIPASOULA                  1

[312112 rows x 3 columns]


## Get the prix meter carre par annee et ville

In [12]:
query = f"""
SELECT EXTRACT(YEAR FROM date_transaction) AS year, 
ville,
prix,
surface_habitable,
prix / surface_habitable AS prix_par_meter
FROM {table_name}
ORDER BY prix_par_meter desc;
"""

prix_par_meter_df = conn.execute(query).fetchdf()

# Print the grouped data
print(f"Nombre de transaction group by year et ville:\n{prix_par_meter_df}")

Nombre de transaction group by year et ville:
         year                   ville      prix  surface_habitable  \
0        2016                FARAMANS   50000.0                  0   
1        2017  SAINT CYR AU MONT D OR  420000.0                  0   
2        2017  SAINT CYR AU MONT D OR  215000.0                  0   
3        2019   SAINT-LAURENT-DE-MURE  207000.0                  0   
4        2019   SAINT-LAURENT-DE-MURE  235000.0                  0   
...       ...                     ...       ...                ...   
9141568  2015    BOULOGNE-BILLANCOURT       0.0                 89   
9141569  2015               MONTROUGE       0.0                 84   
9141570  2015       NEUILLY-SUR-SEINE       0.0                210   
9141571  2015                  ANTONY       0.0                 86   
9141572  2015        LEVALLOIS-PERRET       0.0                 39   

         prix_par_meter  
0                   inf  
1                   inf  
2                   inf  
3        

In [24]:
prix_par_meter_tab_name ="price_par_meter"

query = f"""
CREATE OR REPlACE TEMP TABLE {prix_par_meter_tab_name} AS
SELECT * FROM
(
SELECT EXTRACT(YEAR FROM date_transaction) AS year, 
id_ville,
ville,
prix,
surface_habitable,
prix / surface_habitable AS prix_par_meter
FROM {table_name}
) AS prix_data
WHERE prix > 10000 and prix < 5000000 and surface_habitable > 10;
"""

result = conn.execute(query)



In [25]:
prix_par_meter_df = conn.execute(f"select * from {prix_par_meter_tab_name} order by prix_par_meter desc").fetchdf()

# Print the grouped data
print(f"Prix par metre carre group by year et ville:\n{prix_par_meter_df}")

Prix par metre carre group by year et ville:
         year  id_ville                   ville       prix  surface_habitable  \
0        2017       118                PARIS 18  4700000.0                 16   
1        2020       118                PARIS 18  4350000.0                 15   
2        2015       102                PARIS 02  4600000.0                 16   
3        2024       119            SAINT TROPEZ  4450000.0                 17   
4        2021       102                PARIS 02  4500000.0                 18   
...       ...       ...                     ...        ...                ...   
9069379  2019        19           BARCELONNETTE    12000.0                483   
9069380  2024       116                PARIS 16    10200.0                418   
9069381  2020       224              POULDERGAT    14000.0                657   
9069382  2015       115  DISSAY SOUS COURCILLON    15000.0                867   
9069383  2015       305               SALINDRES   120000.0      

## Read a csv file


In [21]:
fiscaux_tab_name = "fr_fiscaux"
fr_fiscaux_path = (data_path / "foyers_fiscaux.csv").as_posix()
result = conn.execute(f"CREATE OR REPLACE TABLE {fiscaux_tab_name} AS SELECT * FROM read_csv_auto('{fr_fiscaux_path}')").fetchdf()

print(result)

    Count
0  315542


In [23]:
# show the first 5 lines
result = conn.execute(f"select * from {fiscaux_tab_name};").fetchdf()

print(f"french fiscaux dataset:\n {result}")

french fiscaux dataset:
         date departement id_ville                    ville  n_foyers_fiscaux  \
0       2022          01      001  L'Abergement-Clémenciat             466.0   
1       2022          01      002    L'Abergement-de-Varey             143.0   
2       2022          01      004        Ambérieu-en-Bugey            9276.0   
3       2022          01      005      Ambérieux-en-Dombes            1073.0   
4       2022          01      006                  Ambléon              78.0   
...      ...         ...      ...                      ...               ...   
315537  2014         976       13            M'TSANGAMOUJI            1782.0   
315538  2014         976       14                 OUANGANI            2043.0   
315539  2014         976       15                 PAMANDZI            3526.0   
315540  2014         976       16                     SADA            2600.0   
315541  2014         976       17                 TSINGONI            3180.0   

        revenu

In [22]:
show_tables()

Exsiting table list:
              name
0       fr_fiscaux
1          fr_immo
2  price_par_meter
3     sample_table


In [27]:
# join the two tables

query = f"""
SELECT 
    t1.*, 
    t2.*
FROM {prix_par_meter_tab_name} AS t1
JOIN {fiscaux_tab_name} AS t2
    ON t1.year = t2.date  -- Match on year
    AND t1.id_ville = t2.id_ville -- Match on city ID
"""

# Execute the query and fetch results
result = conn.execute(query).fetchdf()

# Print the result
print(result)

In [13]:
conn.close()